In [1]:
import findspark
findspark.find()

findspark.init("/Users/yanji/opt/spark")

import pyspark

spark = pyspark.sql.SparkSession.builder \
    .master("local") \
    .appName("test") \
    .config("spark.driver.bindAddress","192.168.0.164") \
    .config("spark.ui.port","4050") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

print("http://{}:{}".format(spark.conf.get("spark.driver.bindAddress"), spark.conf.get("spark.ui.port")))


22/05/19 15:11:09 WARN Utils: Your hostname, Yans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.133 instead (on interface en0)
22/05/19 15:11:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/19 15:11:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/19 15:11:10 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
22/05/19 15:11:10 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
22/05/19 15:11:10 WARN Utils: Service 'sparkDriver' could not bind on a random free port.

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.net.BindException: Can't assign requested address: Service 'sparkDriver' failed after 16 retries (on a random free port)! Consider explicitly setting the appropriate binding address for the service 'sparkDriver' (for example spark.driver.bindAddress for SparkDriver) to the correct binding address.
	at sun.nio.ch.Net.bind0(Native Method)
	at sun.nio.ch.Net.bind(Net.java:438)
	at sun.nio.ch.Net.bind(Net.java:430)
	at sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:225)
	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:134)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:562)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.bind(DefaultChannelPipeline.java:1334)
	at io.netty.channel.AbstractChannelHandlerContext.invokeBind(AbstractChannelHandlerContext.java:506)
	at io.netty.channel.Abstract

In [10]:
df = spark.read.option("inferSchema", "true").option("header", "true")\
    .csv("../data/flight-data/csv/2015-summary.csv")

In [11]:
df.sort("count").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Estonia|    1|
|              Kosovo|      United States|    1|
|              Zambia|      United States|    1|
|       United States|   Papua New Guinea|    1|
|               Malta|      United States|    1|
|       United States|          Gibraltar|    1|
|            Suriname|      United States|    1|
|       United States|            Croatia|    1|
|            Djibouti|      United States|    1|
|        Burkina Faso|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|             Cyprus|    1|
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
|              Cyprus|      United States|    1|
|       United States|          Lithuania|    1|
|       United States|           Bulgaria|    1|
|       United State

In [12]:
df.createOrReplaceTempView("flight_data_2015")
sql = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1) AS count
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME ORDER BY count DESC
""")
dft = df.groupby("DEST_COUNTRY_NAME").count()

In [13]:
maxsql = spark.sql('''
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY destination_total DESC
'''
)
# maxsql.show()
maxsql.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [destination_total#116L DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(destination_total#116L DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [id=#114]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#83], functions=[sum(count#85)])
         +- Exchange hashpartitioning(DEST_COUNTRY_NAME#83, 200), ENSURE_REQUIREMENTS, [id=#111]
            +- HashAggregate(keys=[DEST_COUNTRY_NAME#83], functions=[partial_sum(count#85)])
               +- FileScan csv [DEST_COUNTRY_NAME#83,count#85] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/yanji/Documents/codes/Spark-The-Definitive-Guide/data/flig..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>




In [14]:
df.groupBy("DEST_COUNTRY_NAME").sum("count").withColumnRenamed("sum(count)", "destination_total") \
    .sort('destination_total', ascending=False).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [destination_total#129L DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(destination_total#129L DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [id=#134]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#83], functions=[sum(count#85)])
         +- Exchange hashpartitioning(DEST_COUNTRY_NAME#83, 200), ENSURE_REQUIREMENTS, [id=#131]
            +- HashAggregate(keys=[DEST_COUNTRY_NAME#83], functions=[partial_sum(count#85)])
               +- FileScan csv [DEST_COUNTRY_NAME#83,count#85] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/yanji/Documents/codes/Spark-The-Definitive-Guide/data/flig..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>


